We will use the dataset from HuggingFace and train it using keras to see which performs better

In [1]:
! pip install datasets
! pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 951.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=a1000a276e408d88566b3b4c474778a3360039def5e9fb0e63e182ab7429ce15
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [3]:
from datasets import list_datasets
from datasets import load_dataset
emotions=load_dataset("emotion")
emotions

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
training=emotions["train"]
testing=emotions["test"]
validating=emotions["validation"]
training[0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [11]:
training_sentences=[]
testing_sentences=[]
validating_sentences=[]
training_labels=[]
testing_labels=[]
validating_labels=[]
for item in training:
  training_sentences.append(item["text"])
  training_labels.append(item["label"])
def conversion_array(containing_array,sentences_array,labels_array):
  for element in containing_array:
    sentences_array.append(element["text"])
    labels_array.append(element["label"])

conversion_array(testing,testing_sentences,testing_labels)
conversion_array(validating,validating_sentences,validating_labels)
testing_sentences[0]
# validating_sentences[0]


'im feeling rather rotten so im not very ambitious right now'

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size=10000
oov_tok='<OOV>'
max_length=100
padding_type='post'
trunc_type='post'
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences) # fit tokenizer only on the training set
word_index= tokenizer.word_index
training_sequences=tokenizer.texts_to_sequences(training_sentences) # create training sequences and pad them
training_padding=pad_sequences(training_sequences,maxlen=max_length,
                               padding=padding_type,truncating=trunc_type)
testing_sequences=tokenizer.texts_to_sequences(testing_sentences) # create testing sequences and pad them
testing_padding=pad_sequences(testing_sequences,maxlen=max_length,
                              padding=padding_type, truncating=trunc_type)
validating_sequences=tokenizer.texts_to_sequences(validating_sentences) # create validating sequences and pad them
validating_padding=pad_sequences(validating_sequences,maxlen=max_length,
                              padding=padding_type, truncating=trunc_type)